In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

import altair as alt
import matplotlib.pyplot as plt

no_seed = 572

In [10]:
df_raw = pd.read_csv('data/milknew_raw.csv')
# df_raw = df_raw.reset_index()
print(df_raw.shape)
df_raw.head(3)

(1059, 8)


,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour,Grade
0,6.6,35,1,0,1,0,254,high
1,6.6,36,0,1,0,1,253,high
2,8.5,70,1,1,1,1,246,low


# EDA

## general: info(), decribe(), count()

In [11]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059 entries, 0 to 1058
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pH          1059 non-null   float64
 1   Temprature  1059 non-null   int64  
 2   Taste       1059 non-null   int64  
 3   Odor        1059 non-null   int64  
 4   Fat         1059 non-null   int64  
 5   Turbidity   1059 non-null   int64  
 6   Colour      1059 non-null   int64  
 7   Grade       1059 non-null   object 
dtypes: float64(1), int64(6), object(1)
memory usage: 66.3+ KB


In [12]:
df_raw.describe()

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour
count,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000
mean,6.630123,44.226629,0.546742,0.432483,0.671388,0.491029,251.840415
std,1.399679,10.098364,0.498046,0.495655,0.469930,0.500156,4.307424
min,3.000000,34.000000,0.000000,0.000000,0.000000,0.000000,240.000000
25%,6.500000,38.000000,0.000000,0.000000,0.000000,0.000000,250.000000
50%,6.700000,41.000000,1.000000,0.000000,1.000000,0.000000,255.000000
75%,6.800000,45.000000,1.000000,1.000000,1.000000,1.000000,255.000000
max,9.500000,90.000000,1.000000,1.000000,1.000000,1.000000,255.000000


In [15]:
df_raw.Grade.value_counts(dropna=False)

low       429
medium    374
high      256
Name: Grade, dtype: int64

## Correlation

In [41]:
df_raw.loc[df_raw['Grade']=='low', 'Grade'] = 0
df_raw.loc[df_raw['Grade']=='medium', 'Grade'] = 1
df_raw.loc[df_raw['Grade']=='high', 'Grade'] = 2

In [42]:
# person
corr = df_raw.corr()
corr.style.background_gradient(cmap='coolwarm', vmin=-1, vmax=1)

/var/folders/8y/pz7j2fcs17q_j4xz5yspd4qc0000gn/T/ipykernel_1495/2095385434.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df_raw.corr()


,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour
pH,1.000000,0.244684,-0.064053,-0.081331,-0.093429,0.048384,-0.164565
Temprature,0.244684,1.000000,-0.109792,-0.048870,0.024073,0.185106,-0.008511
Taste,-0.064053,-0.109792,1.000000,0.017582,0.324149,0.055755,-0.082654
Odor,-0.081331,-0.048870,0.017582,1.000000,0.314505,0.457935,-0.039361
Fat,-0.093429,0.024073,0.324149,0.314505,1.000000,0.329264,0.114151
Turbidity,0.048384,0.185106,0.055755,0.457935,0.329264,1.000000,0.136436
Colour,-0.164565,-0.008511,-0.082654,-0.039361,0.114151,0.136436,1.000000


In [43]:
# spearman for non-linear
corr = df_raw.corr(method='spearman')
corr.style.background_gradient(cmap='coolwarm', vmin=-1, vmax=1)

/var/folders/8y/pz7j2fcs17q_j4xz5yspd4qc0000gn/T/ipykernel_1495/2105138313.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df_raw.corr(method='spearman')


,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour
pH,1.000000,0.341873,-0.033943,0.037125,-0.024233,0.126381,-0.180541
Temprature,0.341873,1.000000,-0.105777,0.119815,0.078101,0.243970,-0.144958
Taste,-0.033943,-0.105777,1.000000,0.017582,0.324149,0.055755,-0.104483
Odor,0.037125,0.119815,0.017582,1.000000,0.314505,0.457935,-0.045223
Fat,-0.024233,0.078101,0.324149,0.314505,1.000000,0.329264,0.056277
Turbidity,0.126381,0.243970,0.055755,0.457935,0.329264,1.000000,0.116935
Colour,-0.180541,-0.144958,-0.104483,-0.045223,0.056277,0.116935,1.000000
